### Why hospital readmissions matter? (for healthcare professionals & for healthcare business)

A hospital readmission is when a patient who is discharged from the hospital, gets re-admitted again within a certain period of time. Hospital readmission rates for certain conditions are now considered an indicator of hospital quality, and also affect the cost of care adversely. 

improve quality of care for patients and reduce healthcare spending by applying payment penalties to hospitals that have more than expected readmission rates for certain conditions

1/ What factors are the strongest predictors of hospital readmission in diabetic patients?

2/ How well can we predict hospital readmission in this dataset with limited features?

### Choosing a dataset
Finding a good dataset is one of the first challenges (besides defining a meaningful question), when trying out machine learning methods. The current state of the healthcare world is such that we can easily find datasets that rich (full of useful information) but dirty (unstructured content or messy schemas) or datasets that are very clean but otherwise sterile in terms of information contained 

### What preprocessing and feature engineering techniques should be applied?
Before we can get to actual modeling, some wrangling with the data is almost always needed. We applied three types of methods here:

1/ Cleaning tasks such as dropping bad data, dealing with missing values.

2/ Modification of existing features e.g. standardization, log transforms etc.

3/ Creation or derivation of new features, usually from existing ones.


#### Dealing with missing values

First we have to see how many missing values are (which were coded as “?” for most variables in the data):


In [ ]:
# for col in df.columns:
#     if df[col].dtype == object:
#          print(col,df[col][df[col] == '?'].count())
# # gender was coded differently so we use a custom count for this one
# print('gender', df['gender'][df['gender'] == 'Unknown/Invalid'].count())


The return will be:

race 2273
weight 98569
payer_code 40256
medical_specialty 49949
diag_1 21
diag_2 358
diag_3 1423
gender 3


#### Now the important part — deciding what to do:

Weight is missing in over 98% records. Owing to the poor interpretability of missing values and little predictive generalizability to other patients, best thing is to just drop it.

Payer code and Medical Specialty of treating physician also have 40–50% missing values. We decided to drop these, but there are other ways too to deal with such missing values.


In [ ]:
# df = df.drop(['weight','payer_code','medical_specialty'], axis = 1)

Primary (diag_1), Secondary (diag_2) and Additional (diag_3) diagnoses were have very few missing values. Technically, if all three are missing, that’s bad data. So we only drop those records where all three diagnoses are missing.

Gender has only 3 missing or invalid values so we decided to drop these records.

Also, one more cleaning step that depends on understanding the data and some common sense: since we are trying to predict readmissions, those patients who died during this hospital admission, have zero probability of readmission. So we should remove those records (discharge_disposition = 11).


In [1]:
# drop_Idx = set(df[(df['diag_1'] == '?') & (df['diag_2'] == '?') & (df['diag_3'] == '?')].index)
# drop_Idx = drop_Idx.union(set(df['gender'][df['gender'] == 'Unknown/Invalid'].index))
# drop_Idx = drop_Idx.union(set(df[df['discharge_disposition_id'] == 11].index))
# new_Idx = list(set(df.index) - set(drop_Idx))
# df = df.iloc[new_Idx]


We also noticed that for two variables (drugs named citoglipton and examide), all records have the same value. So essentially these cannot provide any interpretive or discriminatory information for predicting readmission, and we dropped these columns as well. Technically, this isn’t a missing value problem but rather a missing information problem.


In [2]:
# df = df.drop(['citoglipton', 'examide'], axis = 1)


Service utilization: The data contains variables for number of inpatient (admissions), emergency room visits and outpatient visits for a given patient in the previous one year. These are (crude) measures of how much hospital/clinic services a person has used in the past year. We added these three to create a new variable called service utilization (see figure below). The idea was to see which version gives us better results. Granted, we did not apply any special weighting to the three ingredients of service utilization but we wanted to try something simple at this stage.


In [3]:
# df['service_utilization'] = df['number_outpatient'] + df['number_emergency'] + df['number_inpatient']


Number of medication changes: The dataset contains 23 features for 23 drugs (or combos) which indicate for each of these, whether a change in that medication was made or not during the current hospital stay of patient. Medication change for diabetics upon admission has been shown by previous research to be associated with lower readmission rates. We decided to count how many changes were made in total for each patient, and declared that a new feature. The reasoning here was to both simplify the model and possibly discover a relationship with number of changes regardless of which drug was changed. In python this is done by:


In [4]:
# keys = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'insulin', 'glyburide-metformin', 'tolazamide', 'metformin-pioglitazone','metformin-rosiglitazone', 'glimepiride-pioglitazone', 'glipizide-metformin', 'troglitazone', 'tolbutamide', 'acetohexamide']
# for col in keys:
#     colname = str(col) + 'temp'
#     df[colname] = df[col].apply(lambda x: 0 if (x == 'No' or x == 'Steady') else 1)
# df['numchange'] = 0
# for col in keys:
#     colname = str(col) + 'temp'
#     df['numchange'] = df['numchange'] + df[colname]
#     del df[colname]


To check the result of doing this, we use the value_counts() method which gives a nice tapering distribution:


In [5]:
# df['numchange'].value_counts()
# 0    72868
# 1    25832
# 2     1308
# 3      107
# 4        5
# Name: numchange, dtype: int64
